Copyright (c) MONAI Consortium  
Licensed under the Apache License, Version 2.0 (the "License");  
you may not use this file except in compliance with the License.  
You may obtain a copy of the License at  
&nbsp;&nbsp;&nbsp;&nbsp;http://www.apache.org/licenses/LICENSE-2.0  
Unless required by applicable law or agreed to in writing, software  
distributed under the License is distributed on an "AS IS" BASIS,  
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  
See the License for the specific language governing permissions and  
limitations under the License.

# nnUNet MONAI Bundle

This notebook demonstrates how to create a MONAI Bundle for a trained nnUNet and use it for inference. This is needed when some other application from the MONAI EcoSystem require a MONAI Bundle (MONAI Label, MonaiAlgo for Federated Learning, etc).

This notebook cover the steps to convert a trained nnUNet model to a consumable MONAI Bundle. The nnUNet training is here perfomed using the `nnUNetV2Runner`.

Optionally, the notebook also demonstrates how to use the same nnUNet MONAI Bundle for training a new model. This might be needed in some applications where the nnUNet training needs to be performed through a MONAI Bundle (i.e., Active Learning in MONAI Label, MonaiAlgo for Federated Learning, etc).

## Setup environment

In [ ]:
!python -c "import monai" || pip install -q "monai-weekly[pillow, tqdm]"
!python -c "import matplotlib" || pip install -q matplotlib
!python -c "import nnunetv2" || pip install -q nnunetv2

## Setup imports

In [ ]:
from monai.config import print_config
from monai.apps import DecathlonDataset
import os
import tempfile
from monai.bundle.config_parser import ConfigParser
from monai.apps.nnunet import nnUNetV2Runner
import random
from monai.apps.nnunet.nnunet_bundle import convert_nnunet_to_monai_bundle
import json
from pathlib import Path
import nnunetv2

print_config()

## Setup data directory

You can specify a directory with the `MONAI_DATA_DIRECTORY` environment variable.  
This allows you to save results and reuse downloads.  
If not specified a temporary directory will be used.

In [ ]:
os.environ["MONAI_DATA_DIRECTORY"] = "MONAI/Data"

In [ ]:
directory = os.environ.get("MONAI_DATA_DIRECTORY")
if directory is not None:
    os.makedirs(directory, exist_ok=True)
root_dir = tempfile.mkdtemp() if directory is None else directory
print(root_dir)

## Download Decathlon Spleen Dataset and Generate Data List

To get the Decathlon Spleen dataset and generate the corresponding data list, you can follow the instructions in the [MSD Datalist Generator Notebook](https://github.com/Project-MONAI/tutorials/blob/main/auto3dseg/notebooks/msd_datalist_generator.ipynb)

In [ ]:
DecathlonDataset(root_dir, "Task09_Spleen", "training", download=True)

In [ ]:
dataroot = os.path.join(root_dir, "Task09_Spleen/")

test_dir = os.path.join(dataroot, "imagesTs/")
train_dir = os.path.join(dataroot, "imagesTr/")
label_dir = os.path.join(dataroot, "labelsTr/")

In [ ]:
datalist_json = {"testing": [], "training": []}

In [ ]:
datalist_json["testing"] = [
    {"image": "./imagesTs/" + file} for file in os.listdir(test_dir) if (".nii.gz" in file) and ("._" not in file)
]

In [ ]:
datalist_json["training"] = [
    {"image": "./imagesTr/" + file, "label": "./labelsTr/" + file, "fold": 0}
    for file in os.listdir(train_dir)
    if (".nii.gz" in file) and ("._" not in file)
]  # Initialize as single fold

In [ ]:
random.seed(42)
random.shuffle(datalist_json["training"])

In [ ]:
num_folds = 5
fold_size = len(datalist_json["training"]) // num_folds
for i in range(num_folds):
    for j in range(fold_size):
        datalist_json["training"][i * fold_size + j]["fold"] = i

In [ ]:
datalist_file = Path(root_dir).joinpath("Task09_Spleen", "Task09_Spleen_folds.json")
with open(datalist_file, "w", encoding="utf-8") as f:
    json.dump(datalist_json, f, ensure_ascii=False, indent=4)
print(f"Datalist is saved to {datalist_file}")

## nnUNet Experiment with nnUNetV2Runner

In the following sections, we will use the nnUNetV2Runner to train a model on the spleen dataset from the Medical Segmentation Decathlon.

We first create the Config file for the nnUNetV2Runner:

In [ ]:
nnunet_root_dir = os.path.join(root_dir, "nnUNet")

os.makedirs(nnunet_root_dir, exist_ok=True)

data_src_cfg = os.path.join(nnunet_root_dir, "data_src_cfg.yaml")
data_src = {
    "modality": "CT",
    "dataset_name_or_id": "09",
    "datalist": str(datalist_file),
    "dataroot": os.path.join(root_dir, "Task09_Spleen"),
}

ConfigParser.export_config_file(data_src, data_src_cfg)

In [ ]:
runner = nnUNetV2Runner(
    input_config=data_src_cfg, trainer_class_name="nnUNetTrainer_10epochs", work_dir=nnunet_root_dir
)

In [ ]:
runner.convert_dataset()

In [ ]:
runner.plan_and_process(npfp=2, n_proc=[2, 2, 2])

In [ ]:
runner.train_single_model(config="3d_fullres", fold=0)

## nnUNet MONAI Bundle for Inference

This section is the relevant part of the nnUNet MONAI Bundle for Inference, showing how to use the trained model to perform inference on new data through the use of a MONAI Bundle, wrapping the native nnUNet model and its pre- and post-processing steps.

We first create the MONAI Bundle for the nnUNet model:

In [ ]:
%%bash


python -m monai.bundle init_bundle nnUNetBundle

rm nnUNetBundle/configs/inference.json
mkdir -p nnUNetBundle/src
touch nnUNetBundle/src/__init__.py
which tree && tree nnUNetBundle || true

We then populate the MONAI Bundle with the configuration for inference:

In [ ]:
%%writefile nnUNetBundle/configs/inference.yaml


imports: 
  - $import json
  - $from pathlib import Path
  - $import os
  - $from ignite.contrib.handlers.tqdm_logger import ProgressBar
  - $import shutil
  - $import src


output_dir: "."
bundle_root: "."
data_list_file : "."
data_dir: "."

fold_id: 0
model_name: "model.pt"
prediction_suffix: "prediction"


modality_conf:
  image:
    suffix: ".nii.gz"

#test_data_list: "$src.dataset.get_subfolder_dataset(@data_dir,@modality_conf)"
test_data_list: "$monai.data.load_decathlon_datalist(@data_list_file, is_segmentation=True, data_list_key='testing', base_dir=@data_dir)"
image_modality_keys: "$list(@modality_conf.keys())"
image_key: "image"
image_suffix: "@image_key"

preprocessing:
  _target_: Compose
  transforms:
  - _target_: LoadImaged
    keys: "image"
    ensure_channel_first: True
    image_only: False

test_dataset:
  _target_: Dataset
  data: "$@test_data_list"
  transform: "@preprocessing"

test_loader:
  _target_: DataLoader
  dataset: "@test_dataset"
  batch_size: 1


device: "$torch.device('cuda')"

nnunet_config:
  model_folder: "$@bundle_root + '/models/fold_'+str(@fold_id)"
  model_name: "@model_name"

network_def: "$monai.apps.nnunet.nnunet_bundle.get_nnunet_monai_predictor(**@nnunet_config)"

postprocessing:
  _target_: "Compose"
  transforms:
    #- _target_: Transposed
    #  keys: "pred"
    #  indices:
    #  - 0
    #  - 3
    #  - 2
    #  - 1
    - _target_: SaveImaged
      keys: "pred"
      resample: False
      output_postfix: "@prediction_suffix"
      output_dir: "@output_dir"
      meta_keys: "image_meta_dict"


testing:
  dataloader: "$@test_loader"
  pbar:
    _target_: "ignite.contrib.handlers.tqdm_logger.ProgressBar"
  test_inferer: "$@inferer"

inferer: 
  _target_: "SimpleInferer"

validator:
  _target_: "SupervisedEvaluator"
  postprocessing: "$@postprocessing"
  device: "$@device"
  inferer: "$@testing#test_inferer"
  val_data_loader: "$@testing#dataloader"
  network: "@network_def"
  val_handlers:
  - _target_: "CheckpointLoader"
    load_path: "$@bundle_root+'/models/fold_'+str(@fold_id)+'/'+@model_name"
    load_dict:
      network_weights: '$@network_def.network_weights'
run:
  - "$@testing#pbar.attach(@validator)"
  - "$@validator.run()"


### nnUnet to MONAI Bundle Conversion

Finally, we convert the nnUNet Trained Model to a Bundle-compatible format using the `convert_nnunet_to_monai_bundle` function:

In [ ]:
nnunet_config = {
    "dataset_name_or_id": "009",
    "nnunet_trainer": "nnUNetTrainer_10epochs",
}

bundle_root = "nnUNetBundle"

convert_nnunet_to_monai_bundle(nnunet_config, bundle_root, 0)

You can then inspect the content of the `models` folder to verify that the model has been converted to the MONAI Bundle format.

In [ ]:
%%bash

which tree && tree nnUNetBundle/models

### Test the MONAI Bundle for Inference

The MONAI Bundle for Inference is now ready to be used for inference on new data

In [ ]:
%%bash


BUNDLE_ROOT=nnUNetBundle
MONAI_DATA_DIRECTORY=MONAI/Data

python -m monai.bundle run \
    --config-file $BUNDLE_ROOT/configs/inference.yaml \
    --bundle-root $BUNDLE_ROOT \
    --data_list_file  $MONAI_DATA_DIRECTORY/Task09_Spleen/Task09_Spleen_folds.json \
    --output-dir $BUNDLE_ROOT/pred_output \
    --data_dir $MONAI_DATA_DIRECTORY/Task09_Spleen \
    --logging-file$BUNDLE_ROOT/configs/logging.conf